<a href="https://colab.research.google.com/github/1235357/SakuraLLM-Colab/blob/main/sakura_14b_qwen2_5_v1_0_q6k_gguf_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the audio player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# 挂载Google网盘
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Thu Nov 28 08:49:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                          

In [ ]:
# 安装依赖
!pip install --upgrade pip
!pip install transformers tokenizers
!pip install vllm huggingface-hub flask pyngrok triton torch

In [ ]:

# 运行API后端(vLLM)
# 使用ngrok进行API映射（ngrokToken留空则使用localtunnel）
from huggingface_hub import hf_hub_download

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(8001)
      public_url = ssh_tunnel.public_url
      print('address：'+public_url)
  else:
      public_url = ssh_tunnels[0].public_url
      print('address：'+public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(8001,)).start()
  public_url = "http://localhost:8001"

%cd $ROOT_PATH

# Download the model for the first time
# !HF_ENDPOINT=https://huggingface.co huggingface-cli download SakuraLLM/Sakura-14B-Qwen2.5-v1.0-GGUF --local-dir models --include sakura-14b-qwen2.5-v1.0-q6k.gguf

!RAY_memory_monitor_refresh_ms="0" HF_ENDPOINT=https://huggingface.co OMP_NUM_THREADS=36 VLLM_ATTENTION_BACKEND=XFORMERS vllm serve ./models/sakura-14b-qwen2.5-v1.0-q6k.gguf --tokenizer Qwen/Qwen2.5-14B-Instruct --dtype float16 --api-key token-abc123 --kv-cache-dtype auto --max-model-len 4096 --tensor-parallel-size 1 --gpu-memory-utilization 0.99 --disable-custom-all-reduce --enforce-eager --use-v2-block-manager --disable-log-requests --host 0.0.0.0 --port 8001 --served-model-name "Qwen2.5-14B-Instruct" &

print(f"API is accessible at {public_url}")

address：https://73c4-34-19-36-142.ngrok-free.app
/content/gdrive/MyDrive
2024-11-28 09:06:00.016625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 09:06:00.049319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 09:06:00.059285: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 09:06:00.081757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropri